# Cars196

In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import sys
sys.path.append('../')

from utils import repeat_n_times, get_latex_table, create_image_embeddings
from IPython.display import display

import torch
import numpy as np
from glob import glob
import pandas as pd

In [3]:
# Hyperparameters
num_repetitions = 5
include_models = ["ae"]

# Creating embeddings

In [4]:
# image_paths = sorted(glob("../data/cars196/car_ims/*.jpg"))
# image_ids = [int(p.split("/")[-1].replace(".jpg", "")) for p in image_paths]

# # CLIP
# create_image_embeddings(image_paths, image_ids, "../data/cars196/image_features.pt")

In [5]:
# load image features and labels
# # CLIP
image_features = torch.load("../data/cars196/image_features.pt")
# ImageNet21k ViT
# image_features = torch.load("../data/cars196/imagenet_image_features.pt")

df = pd.read_csv("../data/cars196/Cars196.csv")

We only use the test images from the dataset.

In [6]:
cutoff = 8054
image_features = {i: item for i, item in image_features.items() if i > cutoff}
df = df[cutoff:]

In [7]:
image_features = [image_features[i] for i in sorted(image_features.keys())]
image_features = torch.tensor(np.stack(image_features, axis=0))

In [8]:
image_features.shape

torch.Size([8131, 512])

In [9]:
df

,img_name,class_name,class_index,manufacturer,model,type,year,color
8054,8055,FIAT 500 Abarth 2012,98,FIAT,500 Abarth,Hatchback,2012,black
8055,8056,FIAT 500 Abarth 2012,98,FIAT,500 Abarth,Hatchback,2012,black
8056,8057,FIAT 500 Abarth 2012,98,FIAT,500 Abarth,Hatchback,2012,black
8057,8058,FIAT 500 Abarth 2012,98,FIAT,500 Abarth,Hatchback,2012,black
8058,8059,FIAT 500 Abarth 2012,98,FIAT,500 Abarth,Hatchback,2012,black
...,...,...,...,...,...,...,...,...
16180,16181,smart fortwo Convertible 2012,195,smart,fortwo,Convertible,2012,black
16181,16182,smart fortwo Convertible 2012,195,smart,fortwo,Convertible,2012,other
16182,16183,smart fortwo Convertible 2012,195,smart,fortwo,Convertible,2012,red
16183,16184,smart fortwo Convertible 2012,195,smart,fortwo,Convertible,2012,white


# Experiments

## Car Model

In [10]:
# # Scrapes all car model names from the kbb page
# car_models = pd.read_html("https://www.kbb.com/car-make-model-list/new/view-all/make/")
# all_models = car_models[0]["Make"] + " " + car_models[0][".css-1mzj64w-ContentWrapper{margin:10px 24px 10px 0px;}Model"]
# all_models = all_models.dropna().tolist()
# all_models = list(set(all_models))

In [11]:
all_models = torch.load("../data/cars196/all_models_scraped_kbb_2022-08-17.pt")

In [12]:
labels = df["class_name"].tolist()
texts = [f"a photo of a {m}" for m in all_models]

means, stds = repeat_n_times(num_repetitions, labels, image_features, texts=texts, include_models=include_models)

Creating text features
Run 1
Getting Autoencoder performance


Loss (5308): 9.325789451599121: : 5308it [06:16, 14.08it/s] 


Finished optimization. Best loss (8.2438325881958) achieved after 5208 iterations.
Run 2
Getting Autoencoder performance


Loss (4248): 15.136846542358398: : 4248it [05:24, 13.11it/s]


Finished optimization. Best loss (14.2562894821167) achieved after 4148 iterations.
Run 3
Getting Autoencoder performance


...

Finished optimization. Best loss (9.785943984985352) achieved after 6153 iterations.
Run 4
Getting Autoencoder performance


Loss (5280): 9.558919906616211: : 5280it [04:52, 18.04it/s] 


Finished optimization. Best loss (9.140134811401367) achieved after 5180 iterations.
Run 5
Getting Autoencoder performance


Loss (6185): 8.08443546295166: : 6185it [04:22, 23.54it/s]  


Finished optimization. Best loss (7.946663856506348) achieved after 6085 iterations.


In [13]:
display(means)
display(stds)

,AE
AMI,0.605840
NMI,0.661361
mean_average_precision,0.304860
mean_average_precision_at_r,0.200245
mean_reciprocal_rank,0.744523
precision_at_1,0.638322
r_precision,0.326399


,AE
AMI,0.072550
NMI,0.062130
mean_average_precision,0.070110
mean_average_precision_at_r,0.058482
mean_reciprocal_rank,0.063385
precision_at_1,0.080838
r_precision,0.058369


In [14]:
print(get_latex_table(means, stds, "Cars196, Car Model", "fig:cars196_car_models"))

\begin{table}
\centering
\caption{Cars196, Car Model}
\label{fig:cars196_car_models}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.606 $\pm$ 0.073 \\
\textbf{NMI   } &  0.661 $\pm$ 0.062 \\
\textbf{MAP   } &  0.305 $\pm$ 0.070 \\
\textbf{MAP@R } &  0.200 $\pm$ 0.058 \\
\textbf{MRR   } &  0.745 $\pm$ 0.063 \\
\textbf{Prec@1} &  0.638 $\pm$ 0.081 \\
\textbf{R-Prec} &  0.326 $\pm$ 0.058 \\
\bottomrule
\end{tabular}
\end{table}



/Users/konstantinkobs/code/2022-dml-clip/experiments/../utils.py:215: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return final_results.to_latex(bold_rows=True, escape=False, caption=caption, label=label, column_format=columns_format)


In [15]:
print("### Mean")
print((means*100).to_markdown())
print("### Std")
print((stds*100).to_markdown())

### Mean
|                             |      AE |
|:----------------------------|--------:|
| AMI                         | 60.584  |
| NMI                         | 66.1361 |
| mean_average_precision      | 30.486  |
| mean_average_precision_at_r | 20.0245 |
| mean_reciprocal_rank        | 74.4523 |
| precision_at_1              | 63.8322 |
| r_precision                 | 32.6399 |
### Std
|                             |      AE |
|:----------------------------|--------:|
| AMI                         | 7.25504 |
| NMI                         | 6.21298 |
| mean_average_precision      | 7.01105 |
| mean_average_precision_at_r | 5.84819 |
| mean_reciprocal_rank        | 6.3385  |
| precision_at_1              | 8.08381 |
| r_precision                 | 5.83694 |


## Car Manufacturer

In [16]:
# # Scrapes all car model names from the kbb page
# car_models = pd.read_html("https://www.kbb.com/car-make-model-list/new/view-all/make/")
# manufacturers = car_models[0]["Make"]
# manufacturers = manufacturers.dropna().tolist()
# manufacturers = list(set(manufacturers))

In [17]:
# torch.save(manufacturers, "../data/cars196/manufacturers_scraped_kbb_2022-08-17.pt")

In [18]:
manufacturers = torch.load("../data/cars196/manufacturers_scraped_kbb_2022-08-17.pt")

In [19]:
len(manufacturers), df["manufacturer"].nunique()

(46, 35)

In [20]:
labels = df["manufacturer"].tolist()
texts = [f"a photo of a car produced by {m}" for m in manufacturers]

means, stds = repeat_n_times(num_repetitions, labels, image_features, texts=texts, include_models=include_models)

Creating text features
Run 1
Getting Autoencoder performance


Loss (4350): 0.06707895547151566: : 4350it [00:44, 97.68it/s]  


Finished optimization. Best loss (0.04315154254436493) achieved after 4250 iterations.
Run 2
Getting Autoencoder performance


Loss (3595): 0.07832060754299164: : 3595it [00:35, 100.38it/s] 


Finished optimization. Best loss (0.0230050478130579) achieved after 3495 iterations.
Run 3
Getting Autoencoder performance


...

Finished optimization. Best loss (0.018377671018242836) achieved after 4811 iterations.
Run 4
Getting Autoencoder performance


Loss (4383): 0.05439429357647896: : 4383it [00:46, 93.80it/s]  


Finished optimization. Best loss (0.036721475422382355) achieved after 4283 iterations.
Run 5
Getting Autoencoder performance


Loss (4394): 0.0457974411547184: : 4394it [00:47, 93.31it/s]   


Finished optimization. Best loss (0.023741450160741806) achieved after 4294 iterations.


In [21]:
display(means)
display(stds)

,AE
AMI,0.435673
NMI,0.448530
mean_average_precision,0.292637
mean_average_precision_at_r,0.179695
mean_reciprocal_rank,0.737193
precision_at_1,0.631214
r_precision,0.309417


,AE
AMI,0.026471
NMI,0.025865
mean_average_precision,0.024229
mean_average_precision_at_r,0.021859
mean_reciprocal_rank,0.030217
precision_at_1,0.038777
r_precision,0.021342


In [22]:
print(get_latex_table(means, stds, "Cars196, Manufacturer", "fig:cars196_manufacturer"))

\begin{table}
\centering
\caption{Cars196, Manufacturer}
\label{fig:cars196_manufacturer}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.436 $\pm$ 0.026 \\
\textbf{NMI   } &  0.449 $\pm$ 0.026 \\
\textbf{MAP   } &  0.293 $\pm$ 0.024 \\
\textbf{MAP@R } &  0.180 $\pm$ 0.022 \\
\textbf{MRR   } &  0.737 $\pm$ 0.030 \\
\textbf{Prec@1} &  0.631 $\pm$ 0.039 \\
\textbf{R-Prec} &  0.309 $\pm$ 0.021 \\
\bottomrule
\end{tabular}
\end{table}



/Users/konstantinkobs/code/2022-dml-clip/experiments/../utils.py:215: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return final_results.to_latex(bold_rows=True, escape=False, caption=caption, label=label, column_format=columns_format)


In [23]:
print("### Mean")
print(means.to_markdown())
print("### Std")
print(stds.to_markdown())

### Mean
|                             |       AE |
|:----------------------------|---------:|
| AMI                         | 0.435673 |
| NMI                         | 0.44853  |
| mean_average_precision      | 0.292637 |
| mean_average_precision_at_r | 0.179695 |
| mean_reciprocal_rank        | 0.737193 |
| precision_at_1              | 0.631214 |
| r_precision                 | 0.309417 |
### Std
|                             |        AE |
|:----------------------------|----------:|
| AMI                         | 0.0264714 |
| NMI                         | 0.0258648 |
| mean_average_precision      | 0.0242294 |
| mean_average_precision_at_r | 0.0218587 |
| mean_reciprocal_rank        | 0.0302169 |
| precision_at_1              | 0.0387769 |
| r_precision                 | 0.0213424 |


## Car Type

In [24]:
df["type"].nunique()

7

In [25]:
labels = df["type"].tolist()
texts = [f"a photo of a {m}" for m in labels]

means, stds = repeat_n_times(num_repetitions, labels, image_features, texts=texts, include_models=include_models)

Creating text features
Run 1
Getting Autoencoder performance


Loss (1017): 0.002144420053809881: : 1017it [00:07, 137.52it/s] 


Finished optimization. Best loss (0.0014160822611302137) achieved after 917 iterations.
Run 2
Getting Autoencoder performance


Loss (934): 0.0017866528360173106: : 934it [00:08, 116.71it/s]


Finished optimization. Best loss (0.001619459013454616) achieved after 834 iterations.
Run 3
Getting Autoencoder performance


...

Finished optimization. Best loss (0.0018698544008657336) achieved after 2731 iterations.
Run 4
Getting Autoencoder performance


Loss (1110): 0.002532477956265211: : 1110it [00:08, 126.67it/s] 


Finished optimization. Best loss (0.0011555328965187073) achieved after 1010 iterations.
Run 5
Getting Autoencoder performance


Loss (1211): 0.0019117393530905247: : 1211it [00:10, 120.08it/s]


Finished optimization. Best loss (0.0012035096297040582) achieved after 1111 iterations.


In [26]:
display(means)
display(stds)

,AE
AMI,0.390225
NMI,0.390951
mean_average_precision,0.421300
mean_average_precision_at_r,0.244207
mean_reciprocal_rank,0.752729
precision_at_1,0.632198
r_precision,0.420223


,AE
AMI,0.032115
NMI,0.032078
mean_average_precision,0.016596
mean_average_precision_at_r,0.015891
mean_reciprocal_rank,0.022292
precision_at_1,0.031231
r_precision,0.015197


In [27]:
print(get_latex_table(means, stds, "Cars196, Car Type", "fig:cars196_car_type"))

\begin{table}
\centering
\caption{Cars196, Car Type}
\label{fig:cars196_car_type}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.390 $\pm$ 0.032 \\
\textbf{NMI   } &  0.391 $\pm$ 0.032 \\
\textbf{MAP   } &  0.421 $\pm$ 0.017 \\
\textbf{MAP@R } &  0.244 $\pm$ 0.016 \\
\textbf{MRR   } &  0.753 $\pm$ 0.022 \\
\textbf{Prec@1} &  0.632 $\pm$ 0.031 \\
\textbf{R-Prec} &  0.420 $\pm$ 0.015 \\
\bottomrule
\end{tabular}
\end{table}



/Users/konstantinkobs/code/2022-dml-clip/experiments/../utils.py:215: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return final_results.to_latex(bold_rows=True, escape=False, caption=caption, label=label, column_format=columns_format)


In [28]:
print("### Mean")
print(means.to_markdown())
print("### Std")
print(stds.to_markdown())

### Mean
|                             |       AE |
|:----------------------------|---------:|
| AMI                         | 0.390225 |
| NMI                         | 0.390951 |
| mean_average_precision      | 0.4213   |
| mean_average_precision_at_r | 0.244207 |
| mean_reciprocal_rank        | 0.752729 |
| precision_at_1              | 0.632198 |
| r_precision                 | 0.420223 |
### Std
|                             |        AE |
|:----------------------------|----------:|
| AMI                         | 0.0321155 |
| NMI                         | 0.0320785 |
| mean_average_precision      | 0.0165955 |
| mean_average_precision_at_r | 0.0158907 |
| mean_reciprocal_rank        | 0.0222916 |
| precision_at_1              | 0.031231  |
| r_precision                 | 0.0151967 |
